In [ ]:
import logging
import warnings

warnings.filterwarnings("ignore")
logging.getLogger("pytensor").setLevel(logging.ERROR)

# Structural Shocks and Their Effects

A reduced-form VAR tells you that variables are correlated, but not *why*. Structural
identification imposes economic assumptions -- typically about the contemporaneous
causal ordering -- to decompose reduced-form residuals into interpretable structural
shocks. This tutorial covers Cholesky identification, impulse response functions,
forecast error variance decomposition, and historical decomposition. For more
background, see the [Identification explanation](../explanation/identification.md).

In [ ]:
import numpy as np
import pandas as pd

from litterman import VAR, VARData
from litterman.identification import Cholesky
from litterman.samplers import NUTSSampler

## Setup

We use the same DGP as the previous tutorials -- a VAR(1) with GDP growth, inflation,
and an interest rate. The true coefficient matrix encodes a causal structure: GDP
affects inflation with a lag, and the rate reacts to inflation (a Taylor-rule channel).
This is exactly the kind of structure that Cholesky identification can recover.

In [ ]:
rng = np.random.default_rng(42)
T = 200
n_vars = 3

A_true = np.array([
    [0.6, 0.0, -0.1],
    [0.2, 0.5, 0.0],
    [0.0, 0.15, 0.4],
])

y = np.zeros((T, n_vars))
for t in range(1, T):
    y[t] = A_true @ y[t - 1] + rng.standard_normal(n_vars) * 0.1

index = pd.date_range("2000-01-01", periods=T, freq="QS")
data = VARData(endog=y, endog_names=["gdp_growth", "inflation", "rate"], index=index)

sampler = NUTSSampler(draws=500, tune=500, chains=2, cores=1, random_seed=42)
fitted = VAR(lags=1, prior="minnesota").fit(data, sampler=sampler)
fitted

## Cholesky identification

Cholesky identification uses the lower-triangular Cholesky decomposition of the
residual covariance matrix to recover structural shocks. The ordering of variables
determines the causal assumptions:

- **GDP growth** is ordered first -- it is the most exogenous, not contemporaneously
  affected by inflation or rate shocks within the same quarter.
- **Inflation** is ordered second -- it can respond to GDP shocks within the quarter
  but not to rate shocks.
- **Rate** is ordered last -- it can respond contemporaneously to both GDP and
  inflation shocks.

This recursive ordering is a standard assumption in monetary VAR analysis.

In [ ]:
identified = fitted.set_identification_strategy(Cholesky(ordering=["gdp_growth", "inflation", "rate"]))
identified

## Impulse response functions

An impulse response function (IRF) traces how a one-standard-deviation structural
shock propagates through the system over time. Each panel shows the response of one
variable to one shock.

In [ ]:
irf = identified.impulse_response(horizon=20)
fig = irf.plot()

Look at the GDP shock -> inflation panel: a positive GDP shock raises inflation on
impact, with the effect persisting for several quarters before decaying. The rate
shock -> GDP panel shows that a contractionary monetary policy shock (positive rate
shock) reduces GDP growth, as expected from the negative coefficient in the DGP. The
bands represent posterior uncertainty -- wider bands indicate parameters the data are
less informative about.

## Forecast error variance decomposition

The FEVD answers: what fraction of each variable's forecast error variance is
explained by each structural shock at a given horizon? At short horizons, own shocks
dominate. At longer horizons, cross-variable shocks may become more important as
dynamic interdependencies play out.

In [ ]:
fevd = identified.fevd(horizon=20)
fig = fevd.plot()

At horizon 0, each variable's forecast error is entirely explained by its own shock --
a mechanical consequence of the Cholesky decomposition. As the horizon increases, GDP
shocks begin to explain a growing share of inflation's variance, reflecting the lagged
GDP -> inflation channel in the DGP. The rate variable remains largely
self-determined, consistent with its relatively isolated position in the coefficient
matrix.

## Historical decomposition

The historical decomposition attributes the observed value of each variable at each
point in time to the cumulative effect of each structural shock. It answers: which
shocks drove GDP growth above or below its baseline during each quarter?

In [ ]:
hd = identified.historical_decomposition()
fig = hd.plot()

Each panel shows the contribution of each structural shock to a given variable over
the sample period. Large spikes indicate quarters where a particular shock had an
outsized effect. This decomposition is useful for narrative analysis -- identifying
which shocks drove observed macroeconomic outcomes.

## Ordering matters

The Cholesky decomposition is not unique -- different orderings encode different
causal assumptions and produce different structural shocks. Let's re-run the
identification with the rate ordered first (most exogenous) to see how results
change.

In [ ]:
identified_alt = fitted.set_identification_strategy(Cholesky(ordering=["rate", "inflation", "gdp_growth"]))
irf_alt = identified_alt.impulse_response(horizon=20)
fig = irf_alt.plot()

With the rate ordered first, monetary policy shocks are assumed not to respond
contemporaneously to GDP or inflation -- a strong assumption that may not be
realistic. The IRFs change because the structural shocks are different objects under
the two orderings. This sensitivity underscores the importance of choosing an ordering
that reflects genuine economic priors about contemporaneous causation.

For identification schemes that do not rely on a specific ordering, Litterman also
supports sign restrictions via `SignRestriction`.

## Summary

Structural identification transforms a reduced-form VAR into a tool for causal
analysis. The key steps are:

1. **Fit** a reduced-form VAR (see [Quickstart](quickstart.ipynb))
2. **Identify** structural shocks via `set_identification_strategy()`
3. **Analyse** the effects using IRFs, FEVD, and historical decomposition

Each analysis method provides `.plot()`, `.median()`, `.hdi()`, and `.to_dataframe()`
for flexible exploration and export.